In [9]:
import sqlite3
import pandas as pd

In [10]:
def create_table_sql(df, table_name, primary_key=False):
    types_mapping = {
        'object': 'TEXT',
        'int64': 'INTEGER',
        'float64': 'REAL',
        'datetime64[ns]': 'TEXT',
        'bool': 'INTEGER'
    }
    columns_definitions = [f'"{col}" {types_mapping[str(df[col].dtype)]}' for col in df.columns]
    if primary_key:
        columns_definitions.insert(0, '"id" INTEGER PRIMARY KEY AUTOINCREMENT')
    columns = ',\n    '.join(columns_definitions)
    sql = f"CREATE TABLE IF NOT EXISTS {table_name} (\n    {columns}\n);"
    return sql

In [11]:
# Connect to SQLite database
conn = sqlite3.connect('mydatabase.db')

In [12]:
# Read data from CSV files
main_table_df = pd.read_csv(r'Final tables\main-table.csv')
p_outcome_df = pd.read_csv(r'Final tables\p-outcome-table.csv')

In [13]:
# Create SQL tables
cursor = conn.cursor()
cursor.execute(create_table_sql(main_table_df, 'main_table', primary_key=True))
cursor.execute(create_table_sql(p_outcome_df, 'p_outcome_table', primary_key=True))
conn.commit()

In [14]:
# Adjust DataFrame index to start at 101 for primary key
main_table_df.index = range(101, 101 + len(main_table_df))
p_outcome_df.index = range(101, 101 + len(p_outcome_df))

In [ ]:
# Load data into SQLite database from pandas DataFrame
main_table_df.to_sql('main_table', conn, if_exists='replace', index=True, index_label='id')
p_outcome_df.to_sql('p_outcome_table', conn, if_exists='replace', index=True, index_label='id')

In [ ]:
# Example query to verify data import
test_main_df = pd.read_sql_query('SELECT * FROM main_table LIMIT 5', conn)
test_p_outcome_df = pd.read_sql_query('SELECT * FROM p_outcome_table LIMIT 5', conn)
print(test_main_df)
print(test_p_outcome_df)

In [17]:
# Close the database connection
conn.close()